<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/10_%E3%82%B7%E3%83%A5%E3%82%BF%E3%82%A4%E3%83%8A%E3%83%BC%E6%9C%A8%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### シュタイナー木問題 ####################

N,M=map(int,input().split())  #頂点数、辺の数
edge=[]  #無向グラフ（重み付き）　モデル内部で向き（深さ、親子関係）を定義
for _ in range(M):
  u,v,c=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v,c])
  #[v,u,c]を加えてはダメ　全頂点ペアではなく、辺リストを巡回する

U={v-1 for v in map(int,input().split())}  #与えられる頂点部分集合

#バイナリ変数の生成
gen=VariableGenerator()
x_vi=gen.array("Binary",shape=(N,N//2+1))  #頂点vを要素に選んでそれが深さiであるかどうか
x_uvi=gen.array("Binary",shape=(N,N,N//2+1))  #辺(u,v)を要素に選んでそれが深さiであるかどうか
y_v=gen.array("Binary",N)  #頂点vを要素に選ぶかどうか
y_uv=gen.array("Binary",shape=(N,N))  #辺(u,v)を要素に選ぶかどうか

#制約項①　根の存在はただ1つ
f1=0
for v in range(N):
  f1+=x_vi[v][0]
f1=(1-f1)**2

#制約項②　Uに含まれる頂点vはえらぶ必要がある
f2=0
for v in U:
  cnt=0
  for i in range(N//2+1):
    cnt+=x_vi[v][i]
  f2+=(1-cnt)**2

#制約項③　Uに含まれない頂点vについての、各変数の整合性
f3=0
for v in range(N):
  if v in U:
    continue
  cnt=0
  for i in range(N//2+1):
    cnt+=x_vi[v][i]
  f3+=(y_v[v]-cnt)**2

#制約項④　辺の深さと、その両端点の深さには矛盾があってはならない
f4=0
for u,v,c in edge:
  for i in range(1,N//2+1):
    f4+=x_uvi[u][v][i]*(2-x_vi[u][i-1]-x_vi[v][i])
    f4+=x_uvi[v][u][i]*(2-x_vi[v][i-1]-x_vi[u][i])

#制約項⑤　深さiにある頂点には親となる深さi-1の頂点への辺がただ1本存在
f5=0
for v in range(N):
  for i in range(1,N//2+1):
    cnt=0
    for uu,vv,c in edge:
      if vv==v:
          cnt+=x_uvi[uu][vv][i]
    f5+=(x_vi[v][i]-cnt)**2

#制約項⑥　シュタイナー木の要素に使う各辺は1つの深さと向きをもつ
f6=0
for u,v,c in edge:
  cnt=0
  for i in range(1,N//2+1):
    cnt+=x_uvi[u][v][i]-x_uvi[v][u][i]
  f6+=(y_uv[u][v]-cnt)**2

#目的関数項　辺の重みの総和を最小化したい
f7=0
for u,v,c in edge:
  f7+=c*y_uv[u][v]

#ペナルティ係数の設定
A,B=map(int,input().split())
f=A*(f1+f2+f3+f4+f5+f6)+B*f7


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=result.best.values

#シュタイナー木を出力
print(f"エネルギー関数の最小値 = {objective}")
y_uv_values=y_uv.evaluate(values)
chosen=[]
cost=0
for u,v,c in edge:
  if y_uv_values[u][v]==1:
    chosen.append([u+1,v+1,c])
    cost+=c
print(f"辺の重みの総和：{cost}")
print(f"シュタイナー木に含まれる辺：{chosen}")